<a href="https://colab.research.google.com/github/cedro3/CLIP_prefix_caption/blob/main/clip_prefix_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title セットアップ

# install transformer & CLIP
! pip install transformers
! pip install git+https://github.com/openai/CLIP.git

# get code from github
! git clone https://github.com/cedro3/CLIP_prefix_caption.git
%cd CLIP_prefix_caption

# install library
! pip install cog
! pip install redis

# define function
from predict import *
from function import *

# CLIP model + GPT2 tokenizer
clip_model, preprocess = clip.load("ViT-B/32", device='cuda', jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# download pretrained_model
! mkdir pretrained_model
! pip install --upgrade gdown
import gdown
gdown.download('https://drive.google.com/uc?id=14pXWwB4Zm82rsDdvbGguLfx9F8aM7ovT', 'pretrained_model/conceptual_weights.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1IdaBtMSvtyzF0ByVaBHtvM0JYSXRExRX', 'pretrained_model/coco_weights.pt', quiet=False)


In [ ]:
#@title モデル選択 
select ='coco' #@param ['coco', 'conceptual']

if select == 'coco':
  model_path = 'pretrained_model/coco_weights.pt'
else:
  model_path = 'pretrained_model/conceptual_weights.pt'

prefix_length = 10
model = ClipCaptionModel(prefix_length)
model.load_state_dict(torch.load(model_path, map_location=CPU)) 
model = model.eval() 
model = model.to('cuda')

In [ ]:
#@title サンプル画像表示
folder = 'images_'+select
display_pic(folder)

In [ ]:
#@title 画像から文の生成
image = '05.jpg' #@param {type:"string"}
image_path = 'images_'+select+'/'+image
use_beam_search = False #@param {type:"boolean"}  

image = io.imread(image_path)
pil_image = PIL.Image.fromarray(image)
#pil_img = Image(filename=UPLOADED_FILE)
display(pil_image)

image = preprocess(pil_image).unsqueeze(0).to('cuda')
with torch.no_grad():
    # if type(model) is ClipCaptionE2E:
    #     prefix_embed = model.forward_image(image)
    # else:
    prefix = clip_model.encode_image(image).to('cuda', dtype=torch.float32)
    prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)
if use_beam_search:
    generated_text_prefix = generate_beam(model, tokenizer, embed=prefix_embed)[0]
else:
    generated_text_prefix = generate2(model, tokenizer, embed=prefix_embed)

print('\n')
print(generated_text_prefix)